In [7]:
import pandas as pd

def df_to_latex_table(df, alphas, k, methods_order=None, metrics_order=None, caption="", label=""):
    """
    将 DataFrame 转为 LaTeX 表格，格式类似于用户提供的例子
    df: 包含 [method, metric, alpha, mean, std]
    alphas: 需要展示的 alpha 列表，如 [0.1,0.2,...,0.9]
    methods_order: 方法排列顺序
    metrics_order: 指标排列顺序
    """
    # 排序
    if methods_order is None:
        methods_order = df["method"].unique()
    if metrics_order is None:
        metrics_order = df["metric"].unique()

    # 开始拼 LaTeX
    latex = []
    latex.append("\\begin{table}[t]")
    latex.append("\\centering")
    latex.append("\\small")
    latex.append("\\setlength{\\tabcolsep}{6pt}")
    latex.append("\\renewcommand{\\arraystretch}{1.2}")
    latex.append("\\resizebox{0.95\\linewidth}{!}{%")
    latex.append("\\begin{tabular}{@{\\hspace{.6em}}cl" + "c" * len(alphas) + "@{\\hspace{.6em}}}")
    latex.append("\\toprule")

    # 表头
    alpha_headers = " & ".join([f"$\\alpha{{=}}{a}$" for a in alphas])
    latex.append("\\textbf{Method} & \\textbf{Metric} & " + alpha_headers + " \\\\")
    latex.append("\\midrule")

    for mi, method in enumerate(methods_order):
        for j, metric in enumerate(metrics_order):
            row = [("\\multirow{"+str(len(metrics_order))+"}{*}{"+method+"}" if j == 0 else ""), metric]
            for a in alphas:
                sub = df[(df["method"] == method) & (df["metric"] == metric) & (df["alpha"] == a)]
                if not sub.empty:
                    mean, std = sub["mean"].values[0], sub["std"].values[0]
                    cell = f"\\msl{{{mean:.2f}}}{{{std:.2f}}}"
                else:
                    cell = "-"
                row.append(cell)
            latex.append(" & ".join(row) + " \\\\")
        if mi != len(methods_order)-1:
            latex.append("\\cmidrule(lr){1-"+str(2+len(alphas))+"}")

    latex.append("\\bottomrule")
    latex.append("\\end{tabular}")
    latex.append("}% end resizebox")
    latex.append(f"\\caption{{\\small {caption}}}")
    latex.append(f"\\label{{{label}}}")
    latex.append("\\vspace{-1.5\\baselineskip}")
    latex.append("\\end{table}")

    return "\n".join(latex)


In [8]:
import json
import pandas as pd


def get_df(k):
    # === 1. 读取json ===
    with open("bit.json", "r") as f:
        raw_data = json.load(f)

    records = []
    for run_name, info in raw_data.items():
        # === 2. 自定义条件筛选 ===
        if info.get("step", None) != 49999:
            continue
        if info.get("k", None) != k:
            continue
        # 你也可以加别的条件，比如只保留特定 objective
        # if info.get("objective") not in ["db", "fl-db", "tb", "subtb"]:
        #     continue

        alpha_init = round(float(info.get("alpha_init", -1)), 1)
        method = info.get("objective", "")
        # 映射成表格里用的名字
        if method == "db":
            method = "DB"
        elif method == "fl-db":
            method = "FL-DB"
        elif method == "tb":
            method = "TB"
        elif method == "subtb":
            method = "SubTB($\\lambda$)"
        elif method == "fl-subtb":
            method = "FL-SubTB($\\lambda$)"

        records.append({
            "method": method,
            "alpha": alpha_init,
            "modes": info.get("modes", None),
            "spearman": info.get("spearman_corr_test", None)
        })

    df_raw = pd.DataFrame(records)

    # === 3. 分组计算 mean/std ===
    agg_list = []
    for metric, col in [("Modes", "modes"), ("Spearman", "spearman")]:
        grouped = df_raw.groupby(["method", "alpha"])[col].agg(["mean", "std"]).reset_index()
        grouped = grouped.rename(columns={"mean": "mean", "std": "std"})
        grouped["metric"] = metric
        grouped = grouped[["method", "metric", "alpha", "mean", "std"]]
        agg_list.append(grouped)

    df_final = pd.concat(agg_list, ignore_index=True)
    return df_final

alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
k=2
df_final = get_df(k)
# 假设 df 已经有 method, metric, alpha, mean, std
latex_code = df_to_latex_table(df_final, alphas, k,
                               methods_order=["DB","SubTB($\\lambda$)","TB"],
                               metrics_order=["Modes","Spearman"],
                               caption=f"Results on MODES and SPEARMAN\\_CORR across different $\\alpha$ at $k={k}$.",
                               label="tab:num-modes-spearman")

print(latex_code)

\begin{table}[t]
\centering
\small
\setlength{\tabcolsep}{6pt}
\renewcommand{\arraystretch}{1.2}
\resizebox{0.95\linewidth}{!}{%
\begin{tabular}{@{\hspace{.6em}}clccccccccc@{\hspace{.6em}}}
\toprule
\textbf{Method} & \textbf{Metric} & $\alpha{=}0.1$ & $\alpha{=}0.2$ & $\alpha{=}0.3$ & $\alpha{=}0.4$ & $\alpha{=}0.5$ & $\alpha{=}0.6$ & $\alpha{=}0.7$ & $\alpha{=}0.8$ & $\alpha{=}0.9$ \\
\midrule
\multirow{2}{*}{DB} & Modes & \msl{1.80}{0.84} & \msl{0.60}{0.55} & \msl{1.60}{1.52} & \msl{1.40}{1.14} & \msl{0.60}{0.89} & \msl{2.00}{1.00} & \msl{1.00}{0.71} & \msl{2.00}{1.58} & \msl{2.60}{2.07} \\
 & Spearman & \msl{0.51}{0.02} & \msl{0.43}{0.12} & \msl{0.45}{0.13} & \msl{0.48}{0.10} & \msl{0.32}{0.43} & \msl{0.49}{0.04} & \msl{0.40}{0.27} & \msl{-0.15}{0.41} & \msl{0.50}{0.04} \\
\cmidrule(lr){1-11}
\multirow{2}{*}{SubTB($\lambda$)} & Modes & \msl{20.80}{11.39} & \msl{18.20}{14.75} & \msl{13.00}{8.46} & \msl{16.40}{11.67} & \msl{22.80}{13.27} & \msl{14.40}{11.78} & \msl{9.00}{3.67} & \msl{

In [8]:
for k in [2,4,6,8,10]:
    df_final = get_df(k)
    latex_code = df_to_latex_table(df_final, alphas, k,
                                   methods_order=["DB","SubTB($\\lambda$)","TB"],
                                      metrics_order=["Modes","Spearman"],
                                      caption=f"Results on MODES and SPEARMAN\\_CORR across different $\\alpha$ at $k={k}$.",
                                        label="tab:num-modes-spearman")
    print(latex_code)
    print()

\begin{table}[t]
\centering
\small
\setlength{\tabcolsep}{6pt}
\renewcommand{\arraystretch}{1.2}
\resizebox{0.95\linewidth}{!}{%
\begin{tabular}{@{\hspace{.6em}}clccccccccc@{\hspace{.6em}}}
\toprule
\textbf{Method} & \textbf{Metric} & $\alpha{=}0.1$ & $\alpha{=}0.2$ & $\alpha{=}0.3$ & $\alpha{=}0.4$ & $\alpha{=}0.5$ & $\alpha{=}0.6$ & $\alpha{=}0.7$ & $\alpha{=}0.8$ & $\alpha{=}0.9$ \\
\midrule
\multirow{2}{*}{DB} & Modes & \msl{1.80}{0.84} & \msl{0.60}{0.55} & \msl{1.60}{1.52} & \msl{1.40}{1.14} & \msl{0.60}{0.89} & \msl{2.00}{1.00} & \msl{1.00}{0.71} & \msl{2.00}{1.58} & \msl{2.60}{2.07} \\
 & Spearman & \msl{0.51}{0.02} & \msl{0.43}{0.12} & \msl{0.45}{0.13} & \msl{0.48}{0.10} & \msl{0.32}{0.43} & \msl{0.49}{0.04} & \msl{0.40}{0.27} & \msl{-0.15}{0.41} & \msl{0.50}{0.04} \\
\cmidrule(lr){1-11}
\multirow{2}{*}{SubTB($\lambda$)} & Modes & - & - & - & - & - & - & - & - & - \\
 & Spearman & - & - & - & - & - & - & - & - & - \\
\cmidrule(lr){1-11}
\multirow{2}{*}{TB} & Modes & \msl{10.40}

In [12]:
import pandas as pd
import json

# === 1. 读取 bit.json ===
with open("bit.json", "r") as f:
    raw_data = json.load(f)

records = []
for run_name, info in raw_data.items():
    if info.get("step", None) != 49999:
        continue
    k = info.get("k", None)
    alpha_init = round(float(info.get("alpha_init", -1)), 1)
    method = info.get("objective", "")
    if method == "db":
        method = "DB"
    elif method == "tb":
        method = "TB"
    elif method == "subtb":
        method = "SubTB($\\lambda$)"
    else:
        continue

    records.append({
        "k": k,
        "method": method,
        "alpha": alpha_init,
        "modes": info.get("modes", None),
        "spearman": info.get("spearman_corr_test", None)
    })

df_raw = pd.DataFrame(records)

# === 2. 定义 Ours 的 α 选择表 ===
ours_alpha = {
    2: {"DB": 0.9, "SubTB($\\lambda$)": 0.1, "TB": 0.9},
    4: {"DB": 0.9, "SubTB($\\lambda$)": 0.2, "TB": 0.8},
    6: {"DB": 0.8, "SubTB($\\lambda$)": 0.1, "TB": 0.8},
    8: {"DB": 0.9, "SubTB($\\lambda$)": 0.2, "TB": 0.1},
    10: {"DB": 0.6, "SubTB($\\lambda$)": 0.2, "TB": 0.4}
}

# === 3. 获取 baseline (α=0.5) 和 ours ===
rows = []
for k in [2,4,6,8,10]:
    for method in ["DB", "SubTB($\\lambda$)", "TB"]:
# for k in [2]:
#     for method in ["SubTB($\\lambda$)"]:
        baseline = df_raw[(df_raw["k"] == k) & (df_raw["method"] == method) & (df_raw["alpha"] == 0.5)]
        ours_alpha_val = ours_alpha[k][method]
        ours = None
        if ours_alpha_val is not None:
            ours = df_raw[(df_raw["k"] == k) & (df_raw["method"] == method) & (df_raw["alpha"] == ours_alpha_val)]

        # 取 modes 和 spearman
        baseline_modes = baseline["modes"].mean() if not baseline.empty else None
        baseline_modes_std = baseline["modes"].std() if not baseline.empty else None
        baseline_spear = baseline["spearman"].mean() if not baseline.empty else None
        baseline_spear_std = baseline["spearman"].std() if not baseline.empty else None
        ours_modes = ours["modes"].mean() if ours is not None and not ours.empty else None
        ours_modes_std = ours["modes"].std() if ours is not None and not ours.empty else None
        ours_spear = ours["spearman"].mean() if ours is not None and not ours.empty else None
        ours_spear_std = ours["spearman"].std() if ours is not None and not ours.empty else None

        rows.append({
            "k": k,
            "method": method,
            "baseline_modes": baseline_modes,
            "baseline_modes_std": baseline_modes_std,
            "ours_modes": ours_modes,
            "ours_modes_std": ours_modes_std,
            "baseline_spear": baseline_spear,
            "baseline_spear_std": baseline_spear_std,
            "ours_spear": ours_spear,
            "ours_spear_std": ours_spear_std
        })

df_compare = pd.DataFrame(rows)

print(df_compare)


     k            method  baseline_modes  baseline_modes_std  ours_modes  \
0    2                DB             0.6            0.894427         2.6   
1    2  SubTB($\lambda$)            22.8           13.274035        20.8   
2    2                TB            12.2            2.387467        16.8   
3    4                DB             9.8            6.496153        12.0   
4    4  SubTB($\lambda$)            35.4            4.159327        40.4   
5    4                TB            38.0            2.738613        41.8   
6    6                DB             4.2            1.923538         5.2   
7    6  SubTB($\lambda$)            20.6            3.361547        22.2   
8    6                TB            21.6            2.701851        23.4   
9    8                DB            36.4            1.816590        44.4   
10   8  SubTB($\lambda$)            58.6            1.673320        58.6   
11   8                TB            58.8            1.303840        59.0   
12  10      

In [13]:
def df_compare_to_latex(df):
    latex = []
    ks = sorted(df["k"].unique())
    methods = ["DB", "SubTB($\\lambda$)", "TB"]
    metrics = [("modes", "Modes"), ("spear", "Spearman")]

    # === 表头 ===
    latex.append(r"\begin{tabular}{ll" + "cc"*len(methods) + "}")
    latex.append(r"\toprule")

    # 第一行: k=2,k=4...
    header1 = [r"\multicolumn{1}{c}{} & \multicolumn{1}{c}{}"]
    for method in methods:
        header1.append(r"& \multicolumn{2}{c}{\textbf{%s}}" % method)
    latex.append(" ".join(header1) + r" \\")
    latex.append(r"\cmidrule(lr){3-4}\cmidrule(lr){5-6}\cmidrule(lr){7-8}")
    
    # 第二行: Baseline Ours
    header2 = [r"\textbf{k} & \textbf{Metric}"]
    for _ in methods:
        header2.append("& Baseline & Ours")
    latex.append(" ".join(header2) + r" \\")
    latex.append(r"\midrule")

    # === 内容 ===
    for k in ks:
        subdf = df[df["k"] == k]
        for i, (m_key, m_label) in enumerate(metrics):
            row = [rf"\multirow{{2}}{{*}}{{{k}}} & " if i == 0 else "& ", m_label]
            for method in methods:
                r = subdf[subdf["method"] == method]
                if r.empty:
                    row.extend(["& -", "& -"])
                    continue
                r = r.iloc[0]
                b_mean, b_std = r[f"baseline_{m_key}"], r[f"baseline_{m_key}_std"]
                o_mean, o_std = r[f"ours_{m_key}"], r[f"ours_{m_key}_std"]

                # print(b_mean, b_std, o_mean, o_std)

                if i == 0 and b_mean > o_mean:
                    row.append(f"& \\msl{{\\textbf{{{b_mean:.2f}}}}}{{{b_std:.2f}}}")
                else:
                    row.append(f"& \\msl{{{b_mean:.2f}}}{{{b_std:.2f}}}")
                if i == 0 and o_mean > b_mean:
                    row.append(f"& \\msl{{\\textbf{{{o_mean:.2f}}}}}{{{o_std:.2f}}}")
                else:
                    row.append(f"& \\msl{{{o_mean:.2f}}}{{{o_std:.2f}}}")
            latex.append(" ".join(row) + r" \\")
        latex.append(r"\midrule")

    latex.append(r"\bottomrule")
    latex.append(r"\end{tabular}")
    return "\n".join(latex)
latex_table = df_compare_to_latex(df_compare)
print(latex_table)

\begin{tabular}{llcccccc}
\toprule
\multicolumn{1}{c}{} & \multicolumn{1}{c}{} & \multicolumn{2}{c}{\textbf{DB}} & \multicolumn{2}{c}{\textbf{SubTB($\lambda$)}} & \multicolumn{2}{c}{\textbf{TB}} \\
\cmidrule(lr){3-4}\cmidrule(lr){5-6}\cmidrule(lr){7-8}
\textbf{k} & \textbf{Metric} & Baseline & Ours & Baseline & Ours & Baseline & Ours \\
\midrule
\multirow{2}{*}{2} &  Modes & \msl{0.60}{0.89} & \msl{\textbf{2.60}}{2.07} & \msl{\textbf{22.80}}{13.27} & \msl{20.80}{11.39} & \msl{12.20}{2.39} & \msl{\textbf{16.80}}{3.77} \\
&  Spearman & \msl{0.32}{0.43} & \msl{0.50}{0.04} & \msl{0.47}{0.06} & \msl{0.51}{0.19} & \msl{0.47}{0.17} & \msl{0.54}{0.00} \\
\midrule
\multirow{2}{*}{4} &  Modes & \msl{9.80}{6.50} & \msl{\textbf{12.00}}{1.58} & \msl{35.40}{4.16} & \msl{\textbf{40.40}}{2.97} & \msl{38.00}{2.74} & \msl{\textbf{41.80}}{5.07} \\
&  Spearman & \msl{0.58}{0.00} & \msl{0.57}{0.00} & \msl{0.58}{0.00} & \msl{0.60}{0.02} & \msl{0.58}{0.00} & \msl{0.58}{0.00} \\
\midrule
\multirow{2}{*}{6} & 

In [17]:
import math
math.sqrt(36**2-31.5**2)

17.428425057933374